<a href="https://colab.research.google.com/github/Grifths/Projeto_CSV_SCAN/blob/main/Projeto_si.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
import os

# Função para importar CSV para um DataFrame
def importar_csv(caminho_entrada):
    """
    Função para importar um arquivo CSV para um DataFrame.

    Parâmetros:
    caminho_entrada (str): Caminho para o arquivo CSV de entrada.

    Retorna:
    pd.DataFrame: DataFrame com os dados do CSV.
    """
    try:
        df = pd.read_csv(caminho_entrada)
        print(f"Arquivo '{caminho_entrada}' importado com sucesso.")
        return df
    except Exception as e:
        print(f"Erro ao importar o arquivo: {e}")
        return None

# Função para fazer upload de arquivos para o Google Colab
def fazer_upload():
    """
    Função para fazer upload de um arquivo para o Google Colab.

    Retorna:
    dict: Dicionário com o arquivo enviado.
    """
    uploaded = files.upload()
    return uploaded

# Função para carregar e concatenar CSVs
def importar_e_concatenar_csv():
    dfs = []  # Lista para armazenar os DataFrames

    while True:
        # Solicita o upload de um novo arquivo
        uploaded_files = fazer_upload()

        # Verifica se um arquivo foi carregado
        if uploaded_files:
            for nome_arquivo in uploaded_files.keys():
                df_temp = importar_csv(nome_arquivo)  # Lê o arquivo
                if df_temp is not None:  # Verifica se o DataFrame foi importado com sucesso
                    dfs.append(df_temp)  # Adiciona à lista

        # Pergunta ao usuário se deseja carregar mais arquivos
        resposta = input("Deseja carregar outro arquivo? (s/n): ").strip().lower()
        if resposta != 's':
            break

    # Concatena todos os DataFrames em um único DataFrame
    df = pd.concat(dfs, ignore_index=True)
    print(f"Arquivos carregados e concatenados com sucesso.")

    return df

# Chamar a função para importar e concatenar os arquivos CSV
df = importar_e_concatenar_csv()  # O DataFrame final é armazenado na variável df

# Exibir as primeiras linhas do DataFrame para verificação
print(df.head())


In [ ]:
from datetime import datetime


def exportar_csv(df):
    """
    Função para exportar um DataFrame para um arquivo CSV na pasta Downloads com nome gerado automaticamente.

    Parâmetros:
    df (pd.DataFrame): DataFrame a ser exportado.

    Retorna:
    None
    """
    try:
        # Gerar o nome do arquivo automaticamente com base na data e hora
        nome_arquivo = f"ResultadoScan_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

        # Salva o DataFrame temporariamente no Colab
        df.to_csv(nome_arquivo, index=False)

        # Faz o download do arquivo para a máquina do usuário
        files.download(nome_arquivo)
        print(f"Arquivo '{nome_arquivo}' preparado para download.")
    except Exception as e:
        print(f"Erro ao exportar o arquivo: {e}")

In [ ]:
# Função para filtrar o DataFrame mantendo apenas as colunas desejadas
def filtrar_colunas(df):
    """
    Função para filtrar e manter apenas as colunas 'Name', 'Risk' e 'Host'.

    Parâmetros:
    df (pd.DataFrame): DataFrame com os dados.

    Retorna:
    pd.DataFrame: DataFrame com as colunas filtradas.
    """
    # Lista das colunas que você deseja manter
    colunas_desejadas = ['Name', 'Risk', 'Host']

    # Verifica se todas as colunas desejadas estão no DataFrame
    colunas_faltando = [col for col in colunas_desejadas if col not in df.columns]
    if colunas_faltando:
        print(f"As colunas {colunas_faltando} não estão presentes no arquivo.")
        return None

    # Mantém apenas as colunas desejadas
    df_filtrado = df[colunas_desejadas]
    print(f"DataFrame filtrado para manter as colunas: {colunas_desejadas}")

    # Visualiza as primeiras 20 linhas do DataFrame filtrado
    print("Primeiras 20 linhas do DataFrame filtrado:")
    print(df_filtrado.head(20))

    return df_filtrado


In [ ]:
filtrar_colunas(df)

In [ ]:
def remover_duplicatas(df):
    """
    Função para remover duplicatas com base nas combinações de 'Risk', 'Host' e 'Name'.

    Parâmetros:
    df (pd.DataFrame): DataFrame com as colunas 'Risk', 'Host' e 'Name'.

    Retorna:
    pd.DataFrame: DataFrame com as duplicatas removidas.
    """
    # Remover duplicatas considerando todas as colunas
    df_tratado = df.drop_duplicates(subset=['Risk', 'Host', 'Name'], keep='first')

    # Selecionar apenas as colunas desejadas
    df_tratado = df_tratado[['Risk', 'Host', 'Name']]

    return df_tratado

# Exemplo de uso:
# Suponha que 'df' seja o seu DataFrame com os dados
# Aplicar a função para remover duplicatas
df_tratado = remover_duplicatas(df)

# Visualizar o DataFrame tratado
print(df_tratado)

In [ ]:
# Função para transformar os dados no formato desejado
def transformar_para_colunas(df):
    # Criar um dicionário para armazenar os resultados
    resultado = {}

    # Agrupar os dados pelo 'Name'
    for name, group in df.groupby('Name'):
        # Obter o risco associado (assumimos que o risco é o mesmo para todo o grupo)
        risk = group['Risk'].iloc[0]
        # Nome da coluna: "Name - Risk"
        coluna_nome = f"{name} - {risk}"
 # Obter os hosts únicos associados
        hosts = list(set(group['Host']))  # Remove duplicatas e converte de volta para lista
        # Adicionar ao dicionário
        resultado[coluna_nome] = hosts

    # Converter o dicionário em um DataFrame, preenchendo valores ausentes com NaN
    df_final = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in resultado.items()]))

    return df_final

# Aplicar a transformação
df_transformado = transformar_para_colunas(df)

# Exibir o DataFrame transformado
print(df_transformado)

In [ ]:
exportar_csv(df_transformado)

In [ ]:
def calcular_top_criticidade_quantidade(df):
    """
    Calcula o Top 5 das colunas com maior número de Hosts por criticidade
    e exporta o resultado para um CSV, além de somar a quantidade de Hosts por criticidade.

    Parâmetros:
    - df (pd.DataFrame): DataFrame processado com as colunas no formato 'Name - Risk'.

    Retorna:
    - pd.DataFrame: DataFrame com o Top 5 e soma por criticidade.
    """
    # Criar lista para armazenar os resultados
    resultados = []

    for coluna in df.columns:
        # Extrair criticidade da coluna
        nivel_criticidade = coluna.split(' - ')[-1]  # Pegando apenas o nível de criticidade

        # Contar o número de Hosts não nulos na coluna
        quantidade_hosts = df[coluna].notna().sum()

        # Adicionar o resultado
        resultados.append({'Coluna': coluna, 'Criticidade': nivel_criticidade, 'Quantidade de Hosts': quantidade_hosts})

    # Criar DataFrame com os resultados
    df_resultados = pd.DataFrame(resultados)

    # Ordenar pela hierarquia de criticidade e depois pela quantidade de Hosts
    hierarquia_criticidade = ['Critical', 'High', 'Medium', 'Low', 'nan']

    # Mapear criticidade para um valor indexado na hierarquia, com fallback para 'nan'
    def get_criticidade_rank(criticidade):
        if criticidade in hierarquia_criticidade:
            return hierarquia_criticidade.index(criticidade)
        return hierarquia_criticidade.index('nan')

    df_resultados['Criticidade_Rank'] = df_resultados['Criticidade'].apply(get_criticidade_rank)
    df_resultados = df_resultados.sort_values(by=['Criticidade_Rank', 'Quantidade de Hosts'], ascending=[True, False])

    # Selecionar o Top 5
    top_5 = df_resultados.head(5)

    # Calcular o somatório da quantidade de Hosts por criticidade
    soma_criticidade = df_resultados.groupby('Criticidade')['Quantidade de Hosts'].sum().reset_index()
    soma_criticidade = soma_criticidade.sort_values(by='Quantidade de Hosts', ascending=False)

    # Exibir o somatório das criticidades
    print("\nSomatório total da quantidade de Hosts por criticidade:")
    print(soma_criticidade)

    # Salvar o arquivo CSV do Top 5
    top_5.to_csv("top_5_criticidade_quantidade.csv", index=False)
    # Baixar o arquivo Top 5
    files.download("top_5_criticidade_quantidade.csv")

    # Salvar o arquivo CSV do somatório de criticidades
    soma_criticidade.to_csv("soma_criticidade_quantidade.csv", index=False)
    # Baixar o arquivo do somatório
    files.download("soma_criticidade_quantidade.csv")

    return top_5, soma_criticidade

# Chamar a função para calcular o top 5, somar as criticidades e exportar
top_5_resultado, soma_criticidade_resultado = calcular_top_criticidade_quantidade(df_transformado)

# Exibir o Top 5 no console
print("\nTop 5 das colunas por criticidade e quantidade de Hosts:")
print(top_5_resultado)